In [16]:
from neo4j import GraphDatabase
import logging
from neo4j import __version__ as neo4j_version
print(neo4j_version)
import pandas as pd
import h3
from shapely.geometry import Polygon, LineString
import json
from geojson import Feature, Point, FeatureCollection
import geopandas as gpd

5.3.0


In [17]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(
                self.__uri, auth=(self.__user, self.__pwd)
            )
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = (
                self.__driver.session(database=db)
                if db is not None
                else self.__driver.session()
            )
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [18]:
conn = Neo4jConnection(uri="neo4j:localhost:7474", user="neo4j", pwd="password")

In [19]:
## Create graph. ONLY CREATE ONCE!

query_string = f"""
CALL gds.graph.project(
'myGraph',
'H3',
'CAN_PASS',
{{
    relationshipProperties: 'cost'
}}
)
"""


conn.query(query_string)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'myGraph' already exists.}


In [20]:
def shortest_path(from_hex, to_hex):
    query_string = f"""

    MATCH (source:H3 {{hex_name: '{from_hex}'}}), (target:H3 {{hex_name: '{to_hex}'}})
    CALL gds.shortestPath.dijkstra.stream('myGraph', {{
    nodeLabels:['H3'],
    relationshipTypes:['CAN_PASS'],
    relationshipWeightProperty: 'cost',
    sourceNode:source,
    targetNode:target}})
    YIELD path, nodeIds
    RETURN
    [nodeId IN nodeIds | gds.util.asNode(nodeId).hex_name] AS nodeNames,
    nodes(path) as path;

    """
    res = conn.query(query_string)
    res = [record for record in res]
    return res 

In [37]:
#Import sample ports and convert coordiantes to H3 hex ids
file = 'ports.json'

# Opening JSON file
f = open(file)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
ports = [port for port in data]

def reverse(tuples):
    new_tup = tuples[::-1]
    return new_tup

def get_centroid(hex_id):
    boundary = h3.h3_to_geo_boundary(hex_id)
    boundary = [reverse(coord) for coord in boundary]
    return Polygon(boundary).centroid

In [44]:
#Use the first port as origin
origin = h3.geo_to_h3(data[ports[0]]['coordinates'][1], data[ports[0]]['coordinates'][0], 3)

#Calculate shortest path to all ports in JSON
routes = []
for port in ports:
    print(port)
    if 'coordinates' in data[port].keys():
        dest = h3.geo_to_h3(data[port]['coordinates'][1], data[port]['coordinates'][0], 3)
        print(dest)
        if origin != dest:
            print(origin,dest)
            hex_path = shortest_path(origin, dest)
            path = []
            if hex_path:
                for hex in hex_path[0][0]:
                    center = get_centroid(hex)
                    path.append(center)
            else:
                print('not a maritime hex id')
            routes.append({
                'geometry': LineString(path),
                'origin': origin,
                'destination': dest

            })              

AEAJM
8343acfffffffff
AEAUH
8343a0fffffffff
8343acfffffffff 8343a0fffffffff
AEDXB
8343a1fffffffff
8343acfffffffff 8343a1fffffffff
AEFJR
8343a8fffffffff
8343acfffffffff 8343a8fffffffff
AEJEA
8343a1fffffffff
8343acfffffffff 8343a1fffffffff
AEJED
83534dfffffffff
8343acfffffffff 83534dfffffffff
AEKLF
8343acfffffffff
AEPRA
8343acfffffffff
AEQIW
8343acfffffffff
AERKT
8343acfffffffff
AERUW
8343a6fffffffff
8343acfffffffff 8343a6fffffffff
AESHJ
8343acfffffffff
AGANU
835e5dfffffffff
8343acfffffffff 835e5dfffffffff
AGSJO
835e5dfffffffff
8343acfffffffff 835e5dfffffffff
AIBLP
834d4bfffffffff
8343acfffffffff 834d4bfffffffff
AIRBY
834d4bfffffffff
8343acfffffffff 834d4bfffffffff
ALDRZ
831ed4fffffffff
8343acfffffffff 831ed4fffffffff
ANBON
836744fffffffff
8343acfffffffff 836744fffffffff
ANCUR
836746fffffffff
8343acfffffffff 836746fffffffff
ANEUX
835e4bfffffffff
8343acfffffffff 835e4bfffffffff
ANGSB
83e758fffffffff
8343acfffffffff 83e758fffffffff
ANPHI
835e4bfffffffff
8343acfffffffff 835e4bfffffffff
ANSX

CACOL
832b9afffffffff
8343acfffffffff 832b9afffffffff
not a maritime hex id
CACRO
8328dcfffffffff
8343acfffffffff 8328dcfffffffff
CACWL
832b81fffffffff
8343acfffffffff 832b81fffffffff
not a maritime hex id
CADOR
832baafffffffff
8343acfffffffff 832baafffffffff
not a maritime hex id
CADRU
832baafffffffff
8343acfffffffff 832baafffffffff
not a maritime hex id
CAEDM
8312ecfffffffff
8343acfffffffff 8312ecfffffffff
not a maritime hex id
CAHAL
832b0cfffffffff
8343acfffffffff 832b0cfffffffff
CAHAM
832b9bfffffffff
8343acfffffffff 832b9bfffffffff
not a maritime hex id
CAHMC
8328dcfffffffff
8343acfffffffff 8328dcfffffffff
CAKVL
832ab2fffffffff
8343acfffffffff 832ab2fffffffff
not a maritime hex id
CAMAG
832ba8fffffffff
8343acfffffffff 832ba8fffffffff
not a maritime hex id
CAMNT
832b00fffffffff
8343acfffffffff 832b00fffffffff
CAMTR
832baafffffffff
8343acfffffffff 832baafffffffff
not a maritime hex id
CANIA
832b9bfffffffff
8343acfffffffff 832b9bfffffffff
not a maritime hex id
CANNO
8328dcfffffffff
83

CNLSI
834036fffffffff
8343acfffffffff 834036fffffffff
not a maritime hex id
CNLYG
833084fffffffff
8343acfffffffff 833084fffffffff
CNLZH
834152fffffffff
8343acfffffffff 834152fffffffff
not a maritime hex id
CNMAW
8341b5fffffffff
8343acfffffffff 8341b5fffffffff
CNMMI
834109fffffffff
8343acfffffffff 834109fffffffff
CNMWN
83411cfffffffff
8343acfffffffff 83411cfffffffff
CNMYG
8340c4fffffffff
8343acfffffffff 8340c4fffffffff
not a maritime hex id
CNNAH
83411dfffffffff
8343acfffffffff 83411dfffffffff
CNNBO
834b34fffffffff
8343acfffffffff 834b34fffffffff
CNNGB
834b34fffffffff
8343acfffffffff 834b34fffffffff
CNNGG
83411dfffffffff
8343acfffffffff 83411dfffffffff
CNNKG
833091fffffffff
8343acfffffffff 833091fffffffff
not a maritime hex id
CNNNG
834152fffffffff
8343acfffffffff 834152fffffffff
not a maritime hex id
CNNSA
83411dfffffffff
8343acfffffffff 83411dfffffffff
CNNTG
83309dfffffffff
8343acfffffffff 83309dfffffffff
CNPNY
83411dfffffffff
8343acfffffffff 83411dfffffffff
CNPUT
8341a4fffffffff
8343

DKSKA
831f24fffffffff
8343acfffffffff 831f24fffffffff
DKSVE
831f04fffffffff
8343acfffffffff 831f04fffffffff
DMRSU
835e42fffffffff
8343acfffffffff 835e42fffffffff
DOCAU
834cd5fffffffff
8343acfffffffff 834cd5fffffffff
DOHAI
834cd4fffffffff
8343acfffffffff 834cd4fffffffff
DOMAN
834c88fffffffff
8343acfffffffff 834c88fffffffff
DOPOP
834c89fffffffff
8343acfffffffff 834c89fffffffff
DOSDQ
834cd4fffffffff
8343acfffffffff 834cd4fffffffff
DOSPM
834cd5fffffffff
8343acfffffffff 834cd5fffffffff
DZAAE
836a6bfffffffff
8343acfffffffff 836a6bfffffffff
not a maritime hex id
DZALG
833829fffffffff
8343acfffffffff 833829fffffffff
DZBJA
833875fffffffff
8343acfffffffff 833875fffffffff
DZGHA
83380bfffffffff
8343acfffffffff 83380bfffffffff
not a maritime hex id
DZMOS
83382efffffffff
8343acfffffffff 83382efffffffff
DZORN
833823fffffffff
8343acfffffffff 833823fffffffff
DZSKI
833862fffffffff
8343acfffffffff 833862fffffffff
ECBHA
838f2efffffffff
8343acfffffffff 838f2efffffffff
ECCHA
838f2afffffffff
8343acfffffffff 

GBGCI
831862fffffffff
8343acfffffffff 831862fffffffff
GBGLW
831909fffffffff
8343acfffffffff 831909fffffffff
GBGOO
831942fffffffff
8343acfffffffff 831942fffffffff
GBGRG
831972fffffffff
8343acfffffffff 831972fffffffff
GBGRK
831909fffffffff
8343acfffffffff 831909fffffffff
GBGSY
831940fffffffff
8343acfffffffff 831940fffffffff
GBGTY
831941fffffffff
8343acfffffffff 831941fffffffff
GBGVS
83194efffffffff
8343acfffffffff 83194efffffffff
GBHRW
83194efffffffff
8343acfffffffff 83194efffffffff
GBHUL
831940fffffffff
8343acfffffffff 831940fffffffff
GBIMM
831940fffffffff
8343acfffffffff 831940fffffffff
GBINK
831972fffffffff
8343acfffffffff 831972fffffffff
GBINV
83190dfffffffff
8343acfffffffff 83190dfffffffff
GBIPS
83194efffffffff
8343acfffffffff 83194efffffffff
GBIRL
831951fffffffff
8343acfffffffff 831951fffffffff
GBKLN
831943fffffffff
8343acfffffffff 831943fffffffff
GBLAN
831955fffffffff
8343acfffffffff 831955fffffffff
GBLGP
83194efffffffff
8343acfffffffff 83194efffffffff
GBLIV
831951fffffffff
8343ac

...

JPYAT
834b66fffffffff
8343acfffffffff 834b66fffffffff
JPYKK
832e60fffffffff
8343acfffffffff 832e60fffffffff
JPYOK
832f5afffffffff
8343acfffffffff 832f5afffffffff
JPYOS
832f58fffffffff
8343acfffffffff 832f58fffffffff
JPYWT
834b64fffffffff
8343acfffffffff 834b64fffffffff
KEMBA
837b59fffffffff
8343acfffffffff 837b59fffffffff
KENBO
837a6efffffffff
8343acfffffffff 837a6efffffffff
not a maritime hex id
KHKOS
836585fffffffff
8343acfffffffff 836585fffffffff
KHPNH
836584fffffffff
8343acfffffffff 836584fffffffff
not a maritime hex id
KITRW
837ee2fffffffff
8343acfffffffff 837ee2fffffffff
KMMUT
83a322fffffffff
8343acfffffffff 83a322fffffffff
KMYVA
83a335fffffffff
8343acfffffffff 83a335fffffffff
KNBAS
835e4afffffffff
8343acfffffffff 835e4afffffffff
KNNEV
835e4afffffffff
8343acfffffffff 835e4afffffffff
KPCHO
833141fffffffff
8343acfffffffff 833141fffffffff
KPWON
833158fffffffff
8343acfffffffff 833158fffffffff
KRCHA
8330d1fffffffff
8343acfffffffff 8330d1fffffffff
KRCHF
8330c1fffffffff
8343acfffffffff 

NLAMS
831969fffffffff
8343acfffffffff 831969fffffffff
NLBZM
831fa4fffffffff
8343acfffffffff 831fa4fffffffff
NLDOR
831969fffffffff
8343acfffffffff 831969fffffffff
NLEEM
831f32fffffffff
8343acfffffffff 831f32fffffffff
NLGRQ
83196dfffffffff
8343acfffffffff 83196dfffffffff
NLHVH
83196bfffffffff
8343acfffffffff 83196bfffffffff
NLIJM
831968fffffffff
8343acfffffffff 831968fffffffff
NLMOE
831fa4fffffffff
8343acfffffffff 831fa4fffffffff
NLRTM
83196bfffffffff
8343acfffffffff 83196bfffffffff
NLTNZ
83194dfffffffff
8343acfffffffff 83194dfffffffff
NLVEL
831968fffffffff
8343acfffffffff 831968fffffffff
NLVLA
83196bfffffffff
8343acfffffffff 83196bfffffffff
NLVLI
83194dfffffffff
8343acfffffffff 83194dfffffffff
NLZAA
831969fffffffff
8343acfffffffff 831969fffffffff
NLZEW
831969fffffffff
8343acfffffffff 831969fffffffff
NOAAV
830981fffffffff
8343acfffffffff 830981fffffffff
NOAES
830836fffffffff
8343acfffffffff 830836fffffffff
NOAGO
830981fffffffff
8343acfffffffff 830981fffffffff
NOAND
830832fffffffff
8343ac

PTPRV
83351bfffffffff
8343acfffffffff 83351bfffffffff
PTSET
833915fffffffff
8343acfffffffff 833915fffffffff
PTSIE
833915fffffffff
8343acfffffffff 833915fffffffff
PTTER
83351bfffffffff
8343acfffffffff 83351bfffffffff
PTVDC
833922fffffffff
8343acfffffffff 833922fffffffff
PYASU
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
PYBCM
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
PYFNX
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
PYPIL
83a945fffffffff
8343acfffffffff 83a945fffffffff
not a maritime hex id
PYPSE
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
PYSAN
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
PYTER
83a94efffffffff
8343acfffffffff 83a94efffffffff
not a maritime hex id
QADOH
83536bfffffffff
8343acfffffffff 83536bfffffffff
QAMES
83536bfffffffff
8343acfffffffff 83536bfffffffff
QAUMS
83536bfffffffff
8343acfffffffff 83536bfffffffff
RELPT
83a254fffffffff
8343acffffffff

UAODS
831e5dfffffffff
8343acfffffffff 831e5dfffffffff
UASVP
832d28fffffffff
8343acfffffffff 832d28fffffffff
UGJIN
836acdfffffffff
8343acfffffffff 836acdfffffffff
not a maritime hex id
UGKLA
836accfffffffff
8343acfffffffff 836accfffffffff
not a maritime hex id
USADQ
830c5afffffffff
8343acfffffffff 830c5afffffffff
USANC
830c73fffffffff
8343acfffffffff 830c73fffffffff
USANP
832aa8fffffffff
8343acfffffffff 832aa8fffffffff
USAR5
83265afffffffff
8343acfffffffff 83265afffffffff
not a maritime hex id
USATL
8344c1fffffffff
8343acfffffffff 8344c1fffffffff
not a maritime hex id
USBAL
832aa8fffffffff
8343acfffffffff 832aa8fffffffff
USBCC
830c42fffffffff
8343acfffffffff 830c42fffffffff
USBHM
USBNA
83264dfffffffff
8343acfffffffff 83264dfffffffff
not a maritime hex id
USBOS
832a30fffffffff
8343acfffffffff 832a30fffffffff
USBPT
83446efffffffff
8343acfffffffff 83446efffffffff
USBRO
8348b2fffffffff
8343acfffffffff 8348b2fffffffff
USBUF
832aa6fffffffff
8343acfffffffff 832aa6fffffffff
not a maritime hex i

USYOB
8344dafffffffff
8343acfffffffff 8344dafffffffff
not a maritime hex id
UYCYR
83c2d1fffffffff
8343acfffffffff 83c2d1fffffffff
not a maritime hex id
UYMVD
83c2f1fffffffff
8343acfffffffff 83c2f1fffffffff
UYNVP
83c2c5fffffffff
8343acfffffffff 83c2c5fffffffff
UYPDU
83c2c3fffffffff
8343acfffffffff 83c2c3fffffffff
not a maritime hex id
VCKTN
835e71fffffffff
8343acfffffffff 835e71fffffffff
VEAMY
836755fffffffff
8343acfffffffff 836755fffffffff
VEBLA
835f49fffffffff
8343acfffffffff 835f49fffffffff
VEEGU
835f4bfffffffff
8343acfffffffff 835f4bfffffffff
VEETV
83662cfffffffff
8343acfffffffff 83662cfffffffff
VEGUB
83e743fffffffff
8343acfffffffff 83e743fffffffff
VEGUT
835f49fffffffff
8343acfffffffff 835f49fffffffff
VELAG
836741fffffffff
8343acfffffffff 836741fffffffff
VELSV
836752fffffffff
8343acfffffffff 836752fffffffff
VEMAR
836752fffffffff
8343acfffffffff 836752fffffffff
VEMTV
835f41fffffffff
8343acfffffffff 835f41fffffffff
not a maritime hex id
VEPBL
836743fffffffff
8343acfffffffff 836743ffff

In [178]:
gdf = gpd.GeoDataFrame(routes)

In [180]:
with open('routes.geojson' , 'w') as file:
    file.write(gdf.to_json())